# Paso 1: Cargar los Datos (La Solución Mágica)

In [29]:
import pandas as pd

# Reemplaza 'nombre_de_tu_archivo.csv' con el nombre real del archivo que descargaste
# (Asegúrate de que esté en la misma carpeta que tu script)
nombre_archivo = 'se_facturacion_clientes_regulados(in).csv'

try:
    df = pd.read_csv(
        nombre_archivo, 
        delimiter=';',    # La clave: le dice que el separador es ;
        encoding='latin-1'  # La clave 2: arregla los caracteres como 'Ã³'
    )
    
    print("¡Éxito! 🎉 Así se ven tus datos ahora:")
    print(df.head())

except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{nombre_archivo}'.")
except Exception as e:
    print(f"Ocurrió un error: {e}")

¡Éxito! 🎉 Así se ven tus datos ahora:
   ï»¿anio  mes                                             region  \
0     2015    1  RegiÃÂ³n del Libertador Gral. Bernardo OÃ¢â¬...   
1     2015    1  RegiÃÂ³n del Libertador Gral. Bernardo OÃ¢â¬...   
2     2015    1  RegiÃÂ³n del Libertador Gral. Bernardo OÃ¢â¬...   
3     2015    1  RegiÃÂ³n del Libertador Gral. Bernardo OÃ¢â¬...   
4     2015    1  RegiÃÂ³n del Libertador Gral. Bernardo OÃ¢â¬...   

       comuna   tipo_clientes  tarifa  clientes_facturados     e1_kwh  e2_kwh  \
0  Las Cabras  No Residencial   BT2PP                179.0   108809.0     0.0   
1  Las Cabras  No Residencial   AT4.3                128.0  3241469.0     0.0   
2  Las Cabras  No Residencial   AT4.2                 14.0   257092.0     0.0   
3  Las Cabras  No Residencial  BT2PPP                 10.0     6555.0     0.0   
4  Las Cabras  No Residencial   BT4.2                  1.0      588.0     0.0   

   energia_kwh  
0       108809  
1      3241469  
2  

# Paso 2: Limpieza y Filtrado Rápido

In [30]:
# 1. Ver qué tipos de clientes hay
print("Tipos de clientes disponibles:")
print(df['tipo_clientes'].unique())

# 2. Filtrar solo por clientes 'Residencial'
df_limpio = df[df['tipo_clientes'] == 'Residencial'].copy()

print(f"Registros después de filtrar por Residencial: {len(df_limpio)}")

# 3. Primero renombrar la columna con caracteres especiales
# La primera columna tiene un BOM (Byte Order Mark) al inicio
df_limpio = df_limpio.rename(columns={df_limpio.columns[0]: 'anio'})

# 4. Seleccionar solo las columnas que usarán para los modelos
columnas_utiles = [
    'anio',          # Año (predictor X)
    'mes',           # Mes (predictor X)
    'comuna',        # Comuna (predictor X)
    'energia_kwh'    # Consumo (tu variable Y para regresión)
]
df_limpio = df_limpio[columnas_utiles]

# 5. Filtrar valores negativos de energía (errores en los datos)
registros_antes = len(df_limpio)
df_limpio = df_limpio[df_limpio['energia_kwh'] >= 0]
registros_eliminados = registros_antes - len(df_limpio)

print("\nDatos filtrados (solo Residencial y columnas útiles):")
print(df_limpio.head())
print(f"\nNúmero total de registros: {len(df_limpio)}")
if registros_eliminados > 0:
    print(f"⚠️  Se eliminaron {registros_eliminados} registros con valores negativos de energía")

Tipos de clientes disponibles:
['No Residencial' 'Residencial']
Registros después de filtrar por Residencial: 54260

Datos filtrados (solo Residencial y columnas útiles):
    anio  mes       comuna  energia_kwh
8   2015    1  La Estrella       159379
19  2015    1     Litueche       352445
24  2015    1   Las Cabras      1825938
33  2015    1   DoÃÂ±ihue      1128732
39  2015    1     Coltauco       944062

Número total de registros: 54176
⚠️  Se eliminaron 84 registros con valores negativos de energía


# Paso 3: Preparar para los Modelos (¡Obligatorio!)

In [31]:
# 4. Asegurarse de que los números sean números (y no texto)
# 'coerce' convierte errores (si los hay) en 'NaT' o 'NaN' (nulos)
df_limpio['energia_kwh'] = pd.to_numeric(df_limpio['energia_kwh'], errors='coerce')

# 5. Eliminar filas con datos nulos
# (La forma más rápida dada la fecha límite)
df_listo = df_limpio.dropna()

# 6. ¡Crear la variable para CLASIFICACIÓN!
# Esto es OBLIGATORIO para los modelos 2 y 3.
# Usamos 'pd.qcut' para dividir el consumo en 3 grupos (cuantiles)
# del mismo tamaño.
df_listo['Consumo_Categoria'] = pd.qcut(
    df_listo['energia_kwh'],
    q=3,
    labels=['Bajo', 'Medio', 'Alto']
)

print("\n¡DataFrame final listo para los modelos!")
print(df_listo.head())

print("\nRevisión de tipos de datos y nulos:")
df_listo.info()


¡DataFrame final listo para los modelos!
    anio  mes       comuna  energia_kwh Consumo_Categoria
8   2015    1  La Estrella       159379              Bajo
19  2015    1     Litueche       352445             Medio
24  2015    1   Las Cabras      1825938              Alto
33  2015    1   DoÃÂ±ihue      1128732             Medio
39  2015    1     Coltauco       944062             Medio

Revisión de tipos de datos y nulos:
<class 'pandas.core.frame.DataFrame'>
Index: 54176 entries, 8 to 490757
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   anio               54176 non-null  int64   
 1   mes                54176 non-null  int64   
 2   comuna             54176 non-null  object  
 3   energia_kwh        54176 non-null  int64   
 4   Consumo_Categoria  54176 non-null  category
dtypes: category(1), int64(3), object(1)
memory usage: 2.1+ MB
